In [24]:
import pandas as pd
import numpy as np

In [3]:
# pd.read_excel?

In [25]:
allPositions = pd.read_excel('allPositions.xlsx', 'allPositions', skiprows = 1)

In [26]:
allPositions.head()

,证券编码,交易所,证券名称,数量,市值,RM市值,净敞口,多头敞口,空头敞口,CreditSpread,...,产品型1,产品型2,资产类型,行业分类一级,行业分类二级,万德行业分类一级,万德行业分类二级,万德行业分类三级,期权类型,系统源
0,000035.SZ,SZ,中国天楹,17500.0,1.176000e+05,1.176000e+05,1.176000e+05,1.176000e+05,0.0,0.0,...,股票,主板,股票,公用事业,环保工程及服务Ⅱ,工业,商业和专业服务,商业服务与用品,NaN,恒生28
1,000035.SZ,SZ,中国天楹,53000.0,3.561600e+05,3.561600e+05,3.561600e+05,3.561600e+05,0.0,0.0,...,股票,主板,股票,公用事业,环保工程及服务Ⅱ,工业,商业和专业服务,商业服务与用品,NaN,恒生28
2,000046.SZ,SZ,泛海控股,13335849.0,9.388438e+07,9.388438e+07,9.388438e+07,9.388438e+07,0.0,0.0,...,股票,主板,股票,房地产,房地产开发Ⅱ,房地产,房地产Ⅱ,房地产管理和开发,NaN,恒生28
3,000065.SZ,SZ,北方国际,170635.0,2.441787e+06,2.441787e+06,2.441787e+06,2.441787e+06,0.0,0.0,...,股票,主板,股票,建筑装饰,专业工程,工业,资本货物,建筑与工程Ⅲ,NaN,恒生28
4,000333.SZ,SZ,美的集团,1899925.0,1.005250e+08,1.005250e+08,1.005250e+08,1.005250e+08,0.0,0.0,...,股票,主板,股票,家用电器,白色家电,可选消费,耐用消费品与服装,家庭耐用消费品,NaN,恒生28


In [8]:
# allPositions.columns

Index(['证券编码', '交易所', '证券名称', '数量', '市值', 'RM市值', '净敞口', '多头敞口', '空头敞口',
       'CreditSpread', 'Idiosyncratic OAS', 'Issuer OAS', 'Beta', 'Gamma',
       'Delta', 'Theta', 'RHO', 'Vega', 'DV01', '衡泰DV01', '衡泰久期', '久期', '修正久期',
       '修正凸性', 'DOLLAR_DELTA', 'DOLLAR_GAMMA', '95%历史VaR', '99%历史VaR',
       '95%历史增量VaR', '99%历史增量VaR', '95%历史成分VaR', '99%历史成分VaR', '95%历史ESVaR',
       '99%历史ESVaR', '95%蒙特卡罗VaR', '99%蒙特卡罗VaR', '95%蒙卡增量VaR', '99%蒙卡增量VaR',
       '95%蒙卡成分VaR', '99%蒙卡成分VaR', '95%蒙卡ESVaR', '99%蒙卡ESVaR', '是否城投债',
       '是否私募债', '发行人评级', '债券评级', '币种', '国家', '资金账号', '多空头', '套保套利', '分级基金',
       '集团', '公司', '部门', '投资策略', '投资组合', '产品大类1', '产品大类2', '产品型1', '产品型2',
       '资产类型', '行业分类一级', '行业分类二级', '万德行业分类一级', '万德行业分类二级', '万德行业分类三级', '期权类型',
       '系统源'],
      dtype='object')

In [27]:
allPositionsAbbr = allPositions[['证券编码',  '证券名称', '数量', 'RM市值', 'Beta', '部门','产品型1', '资产类型', '行业分类一级']]

In [28]:
stocks = allPositionsAbbr[allPositionsAbbr['产品型1'] == '股票']

In [29]:
stocksA = stocks.dropna() 

In [30]:
stockBeta = stocksA.copy()
stockBeta['capBeta'] = stockBeta['Beta'] * stockBeta['RM市值']

In [16]:
stockBeta.head()

,证券编码,证券名称,数量,RM市值,Beta,部门,产品型1,资产类型,行业分类一级,capBeta
0,000035.SZ,中国天楹,17500.0,1.176000e+05,0.0000,金融衍生品业务总部,股票,股票,公用事业,0.000000e+00
1,000035.SZ,中国天楹,53000.0,3.561600e+05,0.0000,金融衍生品业务总部,股票,股票,公用事业,0.000000e+00
2,000046.SZ,泛海控股,13335849.0,9.388438e+07,-0.0374,金融衍生品业务总部,股票,股票,房地产,-3.511276e+06
3,000065.SZ,北方国际,170635.0,2.441787e+06,1.0286,金融衍生品业务总部,股票,股票,建筑装饰,2.511622e+06
4,000333.SZ,美的集团,1899925.0,1.005250e+08,1.5803,证券投资业务总部,股票,股票,家用电器,1.588597e+08


In [31]:
companyBeta = stockBeta.set_index('行业分类一级').groupby(level = 0)['RM市值', 'capBeta'].agg({'RM市值': np.sum, 'capBeta': np.sum})

In [32]:
industryBeta = companyBeta.copy()
industryBeta['industryBeta'] = industryBeta['capBeta'] / industryBeta['RM市值']

In [33]:
industryBeta

,RM市值,capBeta,industryBeta
行业分类一级,,,
交通运输,9.605582e+07,5.907912e+07,0.615050
休闲服务,2.019956e+08,1.991598e+08,0.985961
传媒,3.570508e+08,5.561506e+08,1.557623
公用事业,8.908588e+07,8.832598e+07,0.991470
农林牧渔,1.165960e+08,1.113303e+08,0.954837
化工,2.964485e+08,2.792803e+08,0.942087
医药生物,1.244350e+09,1.116225e+09,0.897034
商业贸易,1.426507e+08,2.408459e+08,1.688361
国防军工,1.225237e+08,8.365578e+07,0.682772


In [39]:
industryBeta.to_excel('allPositions.xlsx', sheet_name = 'Beta', index=False, header=True)

In [42]:
allPositions.to_excel('allPositions.xlsx', sheet_name='allPositions', index=True, header=True)

In [48]:
writer = pd.ExcelWriter('allPositions.xlsx')
allPositions.to_excel(writer, sheet_name='allPositions', index=True, header=True)
industryBeta.to_excel(writer, sheet_name = 'Beta', index=True, header=True)

In [51]:
writer = pd.ExcelWriter('allPositions.xlsx')
allPositions.to_excel(writer, sheet_name='allPositions', index=False, header=True)
## 输出到sheet'Beta'
industryBeta.to_excel(writer, sheet_name = 'Beta', index=True, header=True)
# 输出到sheet'成份股'
stockA.to_excel(writer, sheet_name = 'comStocks', index=False, header=True)
# 输出到sheet'久期'
bond.to_excel(writer, sheet_name = 'Duration', index=False, header=True)

In [56]:
def abs_duration(x):
	if x <= 3:
		return 3
	if x <= 5:
		return 5
	if x <= 7:
		return 7
	return 'bigThan7'

In [57]:
a = abs_duration(9)

In [58]:
a

'bigThan7'

In [59]:
type(a)

str

In [67]:
def abs_duration(x):
    if x <= 3:
        x = 'y0y3'
    elif x <= 5:
        x = 'y3y5'
    elif x <= 7:
        x = 'y5y7'
    else:
        x = '>y7'
    return x

In [68]:
abs_duration(8)

'>y7'

In [70]:
bond = bond[bond['资产类型'] == '债券'].dropna()
grade = bond.copy()
grade['absDuration'] = np.abs(grade['久期'])
grade.set_index(['absDuration'])

,证券编码,证券名称,数量,RM市值,DV01,久期,资产类型
absDuration,,,,,,,
2.9121,010107.SH,21国债(7),102200.00,1.041929e+07,3075.3357,2.9121,债券
4.4415,010303.SH,03国债(3),543480.00,5.366321e+07,23927.3088,4.4415,债券
4.4415,010303.SH,03国债(3),2052770.00,2.026905e+08,90375.4722,4.4415,债券
6.0141,010504.SH,05国债(4),2230.00,2.287980e+05,137.7957,6.0141,债券
6.0141,010504.SH,05国债(4),90680.00,9.303768e+06,5603.2798,6.0141,债券
2.3364,010512.SH,05国债(12),8100.00,8.117820e+05,189.9386,2.3364,债券
6.8159,010609.SH,06国债(9),500000.00,5.000000e+07,34604.7732,6.8159,债券
0.1153,011754162.IB,17国电SCP007,70000.00,7.001099e+06,83.0403,0.1153,债券
0.2276,011754190.IB,17吉林电力SCP001,500000.00,5.001865e+07,1171.9408,0.2276,债券


In [72]:
def abs_duration(x):
    if x[0] <= 3:
        x[0] = 'y0y3'
    elif x[0] <= 5:
        x[0] = 'y3y5'
    elif x[0] <= 7:
        x[0] = 'y5y7'
    else:
        x[0] = '>y7'
    return x



In [75]:
abc = np.array([[1, 2, 3],
                 [4, 5, 6],
                 [7, 8, 9],
                 [10, 11, 12]])
abc

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [79]:
abc = pd.DataFrame(abc)

In [80]:
abc.apply(abs_duration, axis=1)

,0,1,2
0,y0y3,2,3
1,y3y5,5,6
2,y5y7,8,9
3,>y7,11,12


In [81]:
abc

,0,1,2
0,1,2,3
1,4,5,6
2,7,8,9
3,10,11,12


In [82]:
bond = bond[bond['资产类型'] == '债券'].dropna()
grade = bond.copy()
grade['absDuration'] = np.abs(grade['久期'])

In [83]:
grade.head()

,证券编码,证券名称,数量,RM市值,DV01,久期,资产类型,absDuration
178,010107.SH,21国债(7),102200.0,1.041929e+07,3075.3357,2.9121,债券,2.9121
179,010303.SH,03国债(3),543480.0,5.366321e+07,23927.3088,4.4415,债券,4.4415
180,010303.SH,03国债(3),2052770.0,2.026905e+08,90375.4722,4.4415,债券,4.4415
181,010504.SH,05国债(4),2230.0,2.287980e+05,137.7957,6.0141,债券,6.0141
182,010504.SH,05国债(4),90680.0,9.303768e+06,5603.2798,6.0141,债券,6.0141


In [84]:
def abs_duration(x):
    if x[7] <= 3:
        x[7] = 'y0y3'
    elif x[7] <= 5:
        x[7] = 'y3y5'
    elif x[7] <= 7:
        x[7] = 'y5y7'
    else:
        x[7] = '>y7'
    return x

# abc = np.array[[1, 2, 3],
#                [4, 5, 6],
#                [7, 8, 9],
#                [10, 11, 12]]
# abc.apply(abs_duration, axis = 0)
rating = grade.apply(abs_duration, axis=1)
rating.set_index('absDuration').groupby(level = 0)['RM市值'].agg({'RM市值': np.sum})

C:\Users\xuteng\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


,RM市值
absDuration,
>y7,7.069008e+09
y0y3,3.552502e+10
y3y5,1.971406e+10
y5y7,1.131866e+10


In [86]:
gradeRate = rating.set_index('absDuration').groupby(level = 0)['RM市值'].agg({'RM市值': np.sum})
type(gradeRate)

C:\Users\xuteng\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


pandas.core.frame.DataFrame

In [87]:
gradeRate.to_excel(writer, sheet_name = '久期', index=False, header=True)

In [90]:
writer = pd.ExcelWriter('allPositions.xlsx')
allPositions.to_excel(writer, sheet_name='allPositions', index=False, header=True)
## 输出到sheet'Beta'
industryBeta.to_excel(writer, sheet_name = 'Beta', index=True, header=True)
# 输出到sheet'成份股'
stockA.to_excel(writer, sheet_name = '成份股', index=False, header=True)
# 输出到sheet'评级'
bond.to_excel(writer, sheet_name = '评级', index=False, header=True)
# 输出到sheet'久期'
gradeRate.to_excel(writer, sheet_name = '久期', index=True, header=True)